In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def listArrivee(tempsMax,lam):
    L=[]
    t=0.0
    
    while t<=tempsMax-dt:
        t+=np.random.exponential(lam)
        L.append(t)
    
    return np.array(L)

In [60]:
def addProcess_2S(tmax=300, lama=30, l_lamd=[30], l_nbPers=[0]):
    
    lTmps = [0]
    t = 0.0  # temps du derniere evenement
    assert len(l_lamd) == len(l_nbPers)
    nbService = len(l_lamd)
    # None represente une valeur a definir dans la boucle
    dta = None  # delta inter arrivee
    dtd = nbService * [None]  # deltas inter departs
    cpt = l_nbPers 
    cpt_T = [np.copy(cpt)]  # liste qui accumule les etats des files d'attente a chaque evenement 
        
    while True:
        
        """b_cpt = True
        
        for i in cpt:
            if i != 0:
                b_cpt = False
                
        if b_cpt:
            if dta is None:
                dta = np.random.exponential(lama)
            t += dta            
            cpt[0] += 1
            dta = None"""
        
        # dtdMin = np.argmin(dtd)
        # cptMin = np.argmin(cpt)
        
        if dta is None:
            # la prochaine arrivee n'est pas definie, on la redefinie 
            dta = np.random.exponential(lama)
            
        for i in range(nbService):
            if dtd[i] is None and cpt[i] > 0:
                # le prochain depart n'est definie pour cette file, on le redefini
                dtd[i] = np.random.exponential(l_lamd[i])
        
        # on recherche le delta le plus proche dans le temps
        dtdMin = np.inf
        imin = None
        for i in range(nbService):
            if dtd[i] is not None:
                dtdMin = min(dtd[i],dtdMin)
                imin = i
                
            
        if imin is not None:
            # cas ou l'arrivee est en concurence avec les departs
            if dtd[imin] > dta:
                #l'arrivee se produit avant les desparts
                t += dta
                for i in range(nbService):
                    if dtd[i] is not None:
                        dtd[i] -= dta 
                k = np.random.randint(nbService)  # choix d'une file au hasard
                cpt[k] += 1
                dta = None          
            else:
                # le depart le plus proche se produit avant la prochaine arrivee
                t += dtd[imin]
                dta -= dtd[imin]
                for i in range(nbService):
                    if dtd[i] is not None:
                        dtd[i] -= dtd[imin]
                dtd[imin] = None
                cpt[imin] -= 1
        
        else:
            #cas ou l'arrivee est la seul possibilitee
            t += dta
            k = np.random.randint(nbService)  # choix d'une file au hasard
            cpt[k] += 1
            dta = None  
            
        #verification de si on ne depasse pas les bornes de la simulation
        if t > tmax:
            break

        lTmps.append(t)
        cpt_T.append(np.copy(cpt))
        
    return lTmps,cpt_T  

In [62]:
addProcess_2S(tmax=3000, lama=60, l_lamd=[150,150,150], l_nbPers=[0,0,0])

([0,
  157.27959553367472,
  203.3378077217466,
  221.8706819206473,
  282.0550894806818,
  289.0606140900253,
  318.02604422397997,
  387.6029865259039,
  420.3836889943776,
  550.4821679716731,
  550.7377357223152,
  635.2192404511234,
  637.0519533015878,
  657.5317191443964,
  684.2832810121358,
  750.8459970866487,
  752.9419323085123,
  765.0168549403152,
  798.1890593946495,
  812.3224996315518,
  762.9250384777695,
  834.5999526284818,
  821.1583364642934,
  949.3205203823786,
  1017.9390638761668,
  1134.4841284801744,
  1149.0710916220235,
  1152.8597231148515,
  1265.7235761647291,
  1339.0650010694242,
  1341.2057933943536,
  1346.0762766341948,
  1365.3991443749783,
  1369.7694805888007,
  1418.7728498370943,
  1445.0231324993952,
  1492.875170633593,
  1536.7244291191553,
  1555.848470950652,
  1615.4561686293234,
  1693.9814359231648,
  1707.0953355066383,
  1609.954534147509,
  1708.1688991928374,
  1758.6753279149457,
  1792.4114462679574,
  1508.9425399750428,
  1516.

In [44]:
n

2